# Глава 13 Линейная регрессия (МО Крис Элбон)

# Подгонка прямой

требуется натренировать модель, представляющую линейную связь между признаком и вектором целей

использовать линейную регрессию

In [1]:
#загрузить библиотеки
from sklearn.linear_model import  LinearRegression
from sklearn.datasets import  load_boston

In [2]:
#загрузить данные только с двумя признаками
boston = load_boston()
features = boston.data[:, 0:2]
target = boston.target

In [3]:
#создать объект линейной регрессии
regression = LinearRegression()

In [5]:
#выполнить подгонку линейной модели
model = regression.fit(features, target)

Линейная регрессия исходит из того, что связь между признаками и вектором целей является приблизительно линейной, т е эффект(также называемый коэффициентом, весом или параметром) признаков на вектор целей является постоянным.

В нашем решении для целей объяснения мы натренировали модель, используя всего 2 признака. Наша модель будет иметь вид:

\begin{equation*} \hat{y} = \hat{\beta_o} + \hat{\beta_1}{x_1} + \hat{\beta_2}{x_2} + \epsilon  \end{equation*}

\begin{equation*} \hat{y} \end{equation*} - наша цель

\begin{equation*} {x_i} \end{equation*}  - данные  для одного признака

\begin{equation*} \hat{\beta_1} \end{equation*}  

\begin{equation*} \hat{\beta_2} \end{equation*} - коэффициенты, идентифицированные путем подгонки модели

\begin{equation*} \epsilon \end{equation*} - ошибка

После выполнения подгонки модели, мы можем взглянуть на значение каждого параметра. Например, 

\begin{equation*} \hat{\beta_o} \end{equation*}  

-смещение или пересечение. Можно посмотреть с помощью атрибута intersept_

In [9]:
#взглянуть на точку пересечения
model.intercept_

22.485628113468223

\begin{equation*} \hat{\beta_1} \end{equation*}  
и 
\begin{equation*} \hat{\beta_2} \end{equation*}

можно показать с помощью coef_

In [11]:
#взглянуть на коэффициенты признаков
model.coef_

array([-0.35207832,  0.11610909])

В нашем наборе данных целевым значением является медианное значение жилья в Бостоне в тыс долларов. Поэтому цена первого дома в наборе данных:

In [12]:
#первое значение в векторе целей, умноженное на 1000
target[0]*1000

24000.0

Используя метод predict, мы можем предсказать значение для этого дома

In [13]:
#предсказать целевое значение первого наблюдения, умноженное на 1000
model.predict(features)[0]*1000

24573.366631705547

наша модель сместилась всего на 560 долларов

линейная модель имеет хорошую интерпретируемость:

модельные коэффициенты представляют собой - эффект единичного изменения на вектор целей

Например, первый признак - это количество преступлений на одного жителя.

Коэффициент  равен -0.35

Это значит, что если мы умножим этот коэффициент на 1000(вектором целей является цена дома в тыс долларов), то у нас будет изменение в цене дома для каждого дополнительного преступления на душу населения:

In [14]:
#первый коэффициент, умноженный на 1000
model.coef_[0]*1000

-352.07831564026765

Это говорит о том, что каждое преступление на душу населения снизит цену дома примерно на 350 долларов.

# Обработка интерактивных эффектов

Имеется признак, влияние которого на целевую переменную зависит от другого признака.

Создать член взаимодействия для захвата этой зависимости с помощью  объекта класса PolynomialFeatures библиотеки scikit-learn

In [1]:
#загрузить библиотеки
from sklearn.linear_model import  LinearRegression
from sklearn.datasets import load_boston
from sklearn.preprocessing import PolynomialFeatures

In [2]:
#загрузит данные только с 2 признаками
boston = load_boston()
features = boston.data[:, 0:2]
target = boston.target

In [3]:
#создать член, характеризующий взаимодействие между признаками
interaction = PolynomialFeatures(degree=3, include_bias=False, interaction_only=True)
features_interaction = interaction.fit_transform(features)

In [4]:
#создать объект линейной регрессии
regression = LinearRegression()

In [5]:
#выполнить подгонку линейной регрессии
model = regression.fit(features_interaction, target)

Иногда влияние признака на целевую переменную по крайней мере частично зависит от другого признака. Мы можем включить новый признак, состоящий из произведения соответствующих значений из взаимодействующих признаков.

\begin{equation*} \hat{y} = \hat{\beta_o} + \hat{\beta_1}{x_1} + \hat{\beta_2}{x_2} + \hat{\beta_3}{x_1}{x_2} + \epsilon  \end{equation*}

x1 * x2 - взаимодействие между признаками

In [6]:
#взглянуть на признаки для первого наблюдения
features[0]

array([6.32e-03, 1.80e+01])

Для того, чтобы создать член, характеризующий взаимодействие между признаками, мы просто умножаем эти два значения между собой для каждого наблюдения.

In [7]:
#импортировать библиотеку
import numpy as np

#для каждого наблюдения перемножить значения первого и второго признаков
interaction_term = np.multiply(features[:, 0], features[:, 1])

Затем мы можем взглянуть на член взаимодействия для первого наблюдения.

In [8]:
#взглянуть на член взаимодействия для первого наблюдения
interaction_term[0]

0.11376

Можно использовать класс PolynomialFeatures библиотеки scikit-learn для создания членов взаимодействия для всех сочетаний признаков. Здесь мы можем применить стратегии отбора модели, чтобы определить сочетание признаков и членов взаимодействия, которые производят наилучшую модель.

Для создания членов взаимодействия с использованием объекта PolynomialFeatures 
необходимо задать 3 важных параметра:

interaction_only=True

сообщает объекту PolynomialFeatures возвращать только члены взаимодействия(а не полиноминальные признаки).

include_bias=False

по умолчанию класс PolynomialFeatures добавляет признак, содержащий те, который называется смещением. include_bias=False предотвращает это.

degree

определяет максимальное количество признаков для создания членов взаимодействия (в случае, если мы хотим создать член взаимодействия, который является сочетанием 3-х признаков).

Результат работы объекта класса PolynomialFeatures можно увидеть, проверив соответствуют ли значения признаков первого наблюдения значению члена взаимодействия нашей вручную рассчитанной модели:

In [9]:
#взгрянуть на значения для первого наблюдения
features_interaction[0]

array([6.3200e-03, 1.8000e+01, 1.1376e-01])

In [10]:
features[0]

array([6.32e-03, 1.80e+01])

In [11]:
#для каждого наблюдения перемножить значения первого и второго признаков
interaction_term = np.multiply(features[:, 0], features[:, 1])
#взглянуть на член взаимодействия для первого наблюдения
interaction_term[0]

0.11376

соответствует

# Подгонка нелинейной связи

Требуется смоделировать нелинейную связь

Создать полиноминальную регрессию путем включения полиноминальных признаков и линейную регрессионную модель

In [22]:
#загрузить библиотеки
from sklearn.linear_model import  LinearRegression
from sklearn.datasets import load_boston
from sklearn.preprocessing import  PolynomialFeatures

In [23]:
#загрузить данные с одним признаком
boston = load_boston()
features = boston.data[:, 0:1]
target = boston.target

In [31]:
#создать полиноминальные признаки x^2  и  x^3
polynominal = PolynomialFeatures(degree=3, include_bias=False)
features_polynominal = polynominal.fit_transform(features)

In [32]:
#создать объект линейной регрессии 
regression = LinearRegression()

In [33]:
#выполнить подгонку линейной регрессии
model = regression.fit(features_polynominal, target)

Пример линейной связи: количество этажей и высота здания. 

Многие связи не являются строго линейными.

Пример нелинейной связи:
количество часов, которое учащийся тратит на учебу и оценкой за контрольную работу.

Так оценка учащегося, который вообще не учился и учащийся, который занимался 1 час - будет скорее всего различной. Но существует гораздо меньшая разница в оценках учащегося, который учился в течении 99 часов и учащимся, который учился 100 часов.

Влияние одного часа учебы на итоговую оценку учащегося уменьшается по мере увеличения количества часов.

Полиноминальная  регрессия - это расширение линейной регрессии, позволяющее моделировать нелинейные связи.

Для того, чтобы создать полиноминальную регрессию следует конвертировать линейную функцию:

\begin{equation*} \hat{y} = \hat{\beta_o} + \hat{\beta_1}{x_1}  + \epsilon  \end{equation*}

в полиноминальную, путем добавления полиноминальных признаков. d - степень полинома.

\begin{equation*} \hat{y} = \hat{\beta_o} + \hat{\beta_1}{x_1} + \hat{\beta_2}{x_1^2} +  + \hat{\beta_d}{x_i^d}+ \epsilon  \end{equation*}

Как использовать линейную регрессию, для нелинейного признака? Ответ: мы ничего не изменяем в том, как линейная регрессия подбирает модель, а только добавляем полиноминальные признаки. Т е, линейная регрессия не "знает", что x^2 является квадратичным преобразованием x. Она просто рассматривает его, как еще одну переменную.

С увеличением порядка - увеличивается гибкость модели.

Допестим мы хотим создать полином 3-й степени.  Сосредоточимся только на одном наблюдении(первом наблюдении в наборе данных).

In [34]:
#Вернуть первое наблюдение
features[0]

array([0.00632])

Для того, чтобы создать полиноминальный признак, мы возведем первое значение наблюдения во вторую степень - x1^2

In [35]:
#взглянуть на первое наблюдение, возведенное во вторую степень, x^2
features[0]**2

array([3.99424e-05])

это будет наш новый признак

Затем возводим значение наблюдения в 3-ю степень.

In [36]:
#взглянуть на первое наблюдение, возведенное во третью степень, x^3
features[0]**3

array([2.52435968e-07])

Включив все три признака (x, x^2, x^3) в матрицу признаков, а затем выполнив линейную регрессию, мы провели полиноминальную регрессию.

In [37]:
#взглянуть на значения  первого наблюдения для x, x^2, x^3
features_polynominal[0]

array([6.32000000e-03, 3.99424000e-05, 2.52435968e-07])

Полиноминальные признаки имеют два важных параметра:
degree определяет максимальное число степеней для полиноминальных признаков.
Так, degree=3 будет генерировать x^2 и x^3

По умолчанию объект PolynomialFeatures включает в себя признаки, содержащие одни единицы(называемые смещением). Мы можем удалить их, установив include_bias=False

# Снижение дисперсии с помощью регуляризации

Требуется уменьшить дисперсию линейной регрессионной модели

Использовать обучающий алгоритм, который включает сжимающий штраф(так называемую регуляризацию). Гребневая регрессия и лассо-регрессия.

In [1]:
#загрузить библиотеки
from sklearn.linear_model import Ridge
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler

In [2]:
#загрузить данные
boston = load_boston()
features = boston.data
target = boston.target

In [6]:
#стандартизировать признак
scaler = StandardScaler()
features_standardized = scaler.fit_transform(features)

In [7]:
#создать объект гребневой регрессии со значением альфа
regression = Ridge(alpha=0.5)

In [8]:
#выполнить подгонку линейной регрессии
model = regression.fit(features_standardized, target)

В стандартной линейной регрессии модель тренируется минимизировать сумму квадратической ошибки(погрешности) между истинным y_i и предсказываемыми 
\begin{equation*} \hat{y_i} \end{equation*}
целевыми значениями, или остаточную сумму квадратов RSS

\begin{equation*} RSS = {\sum_{i}^{n} (y_i-\hat{y_i})^2} \end{equation*}

Регуляризованные регрессионные ученики похожи, за исключением того, что они пытаются минимизировать RSS и некий штраф за общий размер значений коэффициентов, называемый сжимающим штрафом, потому-что он пытается сжать модель.

Существует два распространенных типа регуляризационных учеников для линейной регрессии: гребневая регрессия и лассо. Единственным формальным различием между ними является тип применяемого сжимающего штрафа.

В гребневой решрессии сжимающий штраф - это настроечный гиперпараметр, умноженный на  сумму квадратов всех коэффициентов.

\begin{equation*} RSS + {\alpha} {\sum_{j=1}^{p} \hat{\beta_j}^2} \end{equation*}

\begin{equation*} \hat{\beta_j} \end{equation*}

- коэффициент j - го из p признаков

\begin{equation*}  {\alpha}  \end{equation*}

- гиперпараметр

Лассо-регрессия очень похожа на гребневую регрессию, за исключением того, что сжимающий штраф - это настроечный параметр, умножаемый на сумму абсолютных значений всех коэффициентов.

\begin{equation*} \frac{1}{2n} RSS + {\alpha} {\sum_{j=1}^{p} |\hat{\beta_j}|} \end{equation*}

n - количество наблюдений

Так какой же из них следует использовать?
Гребневая регрессия часто дает несколько лучшие предсказания, чем лассо, но лассо производит более интерпретируемы модели.

Если мы хотим сбалансировать штрафные функции между гребнем и лассо, мы можем использовать эластичную сеть, представлющую собой регрессионную модель, в которую вкючены оба штрафа. Независимо от того, какой из них мы используем, гребневая регрессия и лассо регрессия могут штрафовать большие или сложные модели, включая значения коэффициентов в функцию потери, которую мы пытаемся минимизировать.

Гиперпараметр альфа позволяет нам контролировать то, насколько мы штрафуем коэффициенты, где более высокие значения альфа, создают более простые модели.

Идеальное значение альфа должно быть настроено, как и любой другой гиперпараметр. 
В библиотеки scikit-learn  альфа устанавливается с помощью параметра 

\begin{equation*}  {\alpha}  \end{equation*}

Библиотека scikit-learn включает класс RidgeCV , реализующий метод, который позволяет отбирать идеальные значения для альфа

In [9]:
#загрузить библиотеки
from sklearn.linear_model import RidgeCV

In [11]:
#создать объект гребневой регрессии с тремя значениями альфа
regr_cv = RidgeCV(alphas=[0.1, 1, 10])

In [12]:
#выполнить подгонку линейной регрессии
model_cv = regr_cv.fit(features_standardized, target)

In [14]:
#взглянуть на коэффициенты
model_cv.coef_

array([-0.91987132,  1.06646104,  0.11738487,  0.68512693, -2.02901013,
        2.68275376,  0.01315848, -3.07733968,  2.59153764, -2.0105579 ,
       -2.05238455,  0.84884839, -3.73066646])

Мы можем легко просмотреть значение альфа наилучшей модели

In [15]:
#взглянуть на альфа
model_cv.alpha_

1.0

Поскольку в линейной регрессии значение коэффициентов частично определяется шкалой признака, а в регуляризованных моделях все коэффициенты суммируются вместе, перед тренировкой мы должны убедиться, что стандартизировали признак.

# Уменьшение количества признаков с помощью лассо-регрессии

Требуется упростить линейную регрессионную модель, уменьшив количество признаков.

Использовать лассо-регрессию:

In [1]:
#загрузить библиотеки
from sklearn.linear_model import Lasso
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler

In [2]:
#загрузить данные
boston = load_boston()
features = boston.data
target = boston.target

In [4]:
#стандартизировать признаки
scaler = StandardScaler()
features_standardized = scaler.fit_transform(features)

In [5]:
#создать объект лассо регрессии со значением альфа
regression = Lasso(alpha=0.5)

In [6]:
#выполнить подгонку линейной модели
model = regression.fit(features_standardized, target)

Одна из интересных характеристик штрафа лассо-регрессии состоит в том, что он может сжимать коэффициенты модели до нуля, эффективно уменьшая количество признаков в модели.

Например, в нашем решении мы установили альфа на уровне 0.5 и видим, что многие коэффициенты равны нулю. Иными словами, их соответствующие признаки в модели не используются.

In [7]:
#взглянуть на коэффициенты
model.coef_


array([-0.11526463,  0.        , -0.        ,  0.39707879, -0.        ,
        2.97425861, -0.        , -0.17056942, -0.        , -0.        ,
       -1.59844856,  0.54313871, -3.66614361])

Однако, если мы увеличим альфа до гараздо более высокого значения, мы увидим, что буквально ни один из признаков не используется.

In [8]:
#создать ласс регрессию с высоким альфа
regression_a10 = Lasso(alpha=10)
model_a10 = regression_a10.fit(features_standardized, target)
model_a10.coef_

array([-0.,  0., -0.,  0., -0.,  0., -0.,  0., -0., -0., -0.,  0., -0.])

Практическая выгода от этого эффекта: мы можем включить в нашу матрицу признаков 100 признаков, а затем путем настройки гиперпараметра альфа лассо-регрессии создать модель, которая использует только(например) 10 наиболее важных признаков. Это позволяет нам уменьшить дисперсию, улучшая интерпретируемость нашей модели(поскольку меньшее количество признаков легче объяснить).